In [ ]:
import os
import cv2
import numpy as np
from glob import glob
from os.path import join
import time
import math
import operator

In [ ]:
"""
KNN Implementation
"""
def eucledianDistance(desc1, desc2):
    distance = 0
    for x in range(len(desc1)):
        distance += pow(desc1[x] - desc2[x], 2)

    return math.sqrt(distance)


# Eucledian distance matcher, custom/better version
def EUBetter(desc1, desc2, distances, iteration):
    desc1 = np.array(desc1)
    desc2 = np.array(desc2)
    
    distance = desc1 - desc2
    distance = np.power(distance, 2)
    distance = np.sum(distance, axis = 1)
    distance = np.sqrt(distance)
    distance_indices = np.argsort(distance)
    
    k_match = []
    for i in range(2):
        k_match.append(cv2.DMatch(iteration, distance_indices[i], distance[distance_indices[i]]))
    
    return k_match

def KNNMatch(desc1, desc2, k =2): # query, # train
    matches = []
    for i in range(len(desc1)):
        distances = []
        
        t1 = time.time()
        k_match = EUBetter(desc1[i], desc2, distances, i)
        matches.append(k_match)
    
    return matches

In [ ]:
import open3d as o3d

class SfMSolver(object):
    
    def __init__(self, img_pattern, intrinsic, output_dir, downscale = 1):
        """   
        img_pattern: regex pattern used by glob to read the files
        """
        self.img_pattern = img_pattern
        self.K_orig = self.intrinsic_orig = intrinsic.copy()
        self.output_dir = output_dir
        self.downscale = downscale
        self.rescale_intrinsic()
    
    def rescale_intrinsic(self):
        """ 
        if we downscale the image, the intrinsic matrix also need to be changed"""
        
        start = time.time()
        # scale focal length and principal points w.r.t image processing
        if self.downscale > 1:
            self.K = self.K_orig.copy()
            self.K[0, 0] /= float(self.downscale)
            self.K[1, 1] /= float(self.downscale)
            self.K[0, 2] /= float(self.downscale)
            self.K[1, 2] /= float(self.downscale)
            self.intrinsic = self.K
        else:
            self.K = self.intrinsic = self.K_orig.copy()
        elapsed = time.time() - start

    def load_images(self):
        """  
        Loads a set of images to self.imgs list
        """
        start = time.time()
        self.img_paths = sorted(glob(self.img_pattern))
        self.imgs = []
        for idx, img_path in enumerate(self.img_paths):
            try:
                img = cv2.imread(img_path)
                if self.downscale > 1:
                    img = cv2.resized(img, (0, 0), 
                                      fx = 1/float(self.downscale), fy= 1/float(self.downscale),
                                      interpolation = cv2.INTER_LINEAR)
            except Exception as e:
                print("Error loading img: %s" % (img_path))
            
            if img is not None:
                self.imgs.append(img)
                print("loaded img %d size=(%d, %d): %s" % 
                      (idx, img.shape[0], img.shape[1], img_path))
        elapsed = time.time() - start
        
        
    def visualize_matches(self, img1, img2,
                          kp1, kp2, good, 
                          mask = None, save_path = None):
        start = time.time()
        draw_params = dict(matchColor = (0, 255, 0),
                           singlePointColor = None,
                           flags = 2)
        if mask is not None:
            if not isinstance(mask, list):
                matchesMask = mask.ravel().tolist()
            else:
                matchesMask = mask
            draw_matches['matchesMask'] = matchesMask
        img_matches = cv2.drawMatches(
            img1, kp1, img2, kp2, good, None, **draw_params
        )
        cv2.imwrite(save_path, img_matches)
        elapsed = time.time() - start
        
    
    def drawlines(self, img1, img2, lines, pts1, pts2, line_num = None):
        """ 
        Draw line connecting points in two images"""
        start = time.time()
        if img1.ndim ==2:  # Gray
            img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
            img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
            r, c = img1.shape
        else:  # 3
            r, c, _ = img1.shape
        if line_num is not None:
            draw_list = np.random.choice(
                pts1.shape[0], line_num, replace = False
            )
        else:
            draw_list = np.arange(pts1.shape[0])
        
        for ifx, (r, pt1, pt2) in enumerate(zip(lines, pts1, pts2)):
            if idx not in list(draw_list):
                continue
            color = tuple(np.random.randint(0, 255, 3).tolist())
            x0, y0 = map(int, [0, -r[2]/r[1]])
            x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1]])
            
            pt1_int = (int(tuple(pt1.ravel())[0]), int(tuple(pt1.ravel())[1]))
            pt2_int = (int(tuple(pt2.ravel())[0]), int(tuple(pt2.ravel())[1]))
            
            img1 = cv2.line(img1, (int(x0), int(y0)), (int(x1), int(y1)), color, 1)
            img1 = cv2.circle(img1, pt1_int, 5, color, -1)
            img2 = cv2.circle(img2, pt2_int, 5, color, -1)
        
        elapsed = time.time() - start
        
        return img1, img2
    
    
    def visualize_epipolar_lines(self, img1, img2, p1, p2, E, save_path):
        start = time.time()
        # get fundamental matrix
        F, mask_fdm = cv2.findFundamentalMat(p1, p2, cv2.RANSAC)
        p1_selected = p1[mask_fdm.ravel() == 1]
        p2_selected = p2[mask_fdm.ravel() == 1]
        
        # draw lines
        lines1 = cv2.computeCorrespondEpilines(
            p2_selected.reshape(-1, 1, 2), 2, F).reshape(-1, 3)
        img5, _ = self.drawlines(
            img1, img2, lines1, p1_selected, p2_selected, 100
        )
        
        lines2= cv2.computeCorrespondEpilines(
            p1_selected.reshape(-1, 1, 2), 1, F).reshape(-1, 3)
        
        img3, _ = self.drawlines(
            img2, img1, lines2, p2_selected, p1_selected, 100
        )
        canvas = np.concatenate((img5, img3), axis = 1)
        cv2.imwrite(save_path, canvas)
        elapsed = time.time() - start

            